# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for PF coil

In [ ]:
import paramak

pf_coil = paramak. PoloidalFieldCoil(
    height=50,
    width=50,
    center_point=(725, 25),
    name='pf_coil'
)

This makes a 3D geometry and material for the centre column

In [ ]:
center_column = paramak.CenterColumnShieldHyperbola(
    height=1200,
    inner_radius=50,
    mid_radius=100,
    outer_radius=150,
    name='center_column'
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.BlanketConstantThicknessArcH(
    inner_mid_point=(500, 0),
    inner_upper_point=(400, 300),
    inner_lower_point=(400, -300),
    thickness= 100,
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135],
    name='blanket'
)

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, pf_coil,center_column])

my_reactor.show()

At this stage we can export the reactor geometry as stp files and make them avaialbe from download and viewing in FreeCAD.

In [ ]:
my_reactor.export_stp('my_reactor.stp')

from IPython.display import FileLink
display(FileLink('my_reactor.stp'))

Combines and converts the stl file into a DAGMC h5m file that can be used for the model geometry

In [ ]:
my_reactor.export_dagmc_h5m('dagmc.h5m')

import openmc
bound_dag_univ = openmc.DAGMCUniverse(filename='dagmc.h5m').bounded_universe()
geometry = openmc.Geometry(root=bound_dag_univ)

The next section uses materials from the neutronics_material_maker package https://github.com/fusion-energy/neutronics_material_maker/

This has an internal database that knows the isotopes and density of various materials.

The ```.openmc_material``` obtains an openmc.material object

In [ ]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# the materials names are changed after their creation to match the 
# expected material tags in the dagmc file.

mat1 = nmm.Material.from_library(name='copper').openmc_material
mat1.name = 'pf_coil'
mat2 = nmm.Material.from_library(name='Li4SiO4').openmc_material
mat2.name = 'blanket'
mat3 = nmm.Material.from_library(name='copper').openmc_material
mat3.name='center_column'


materials = openmc.Materials([mat1, mat2, mat3])

This section makes the OpenMC geometry from a DAGMC file. This particular DAGMC file does not have a "graveyard" surface at the edge of the geometry so the bound_universe() method is used to automatically add the vacuum surface to the edge of the geometry

In [ ]:

bound_dag_univ = openmc.DAGMCUniverse(filename='dagmc.h5m', auto_geom_ids=True).bounded_universe()
my_geometry = openmc.Geometry(root=bound_dag_univ)

This next step makes a simple point source. This uses native OpenMC commands but one could also use the openmc-plasma-source package https://github.com/fusion-energy/openmc-plasma-source/

In [ ]:

# initialises a new source object
source = openmc.IndependentSource()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

This next section combines the geometry with the materials and specifies a 2D mesh tally that spans the entire geometry

In [ ]:

mesh = openmc.RegularMesh().from_domain(
    my_geometry, # the corners of the mesh are being set automatically to surround the geometry
    dimension=[100, 1, 100] # only 1 cell in the Y dimension
)

mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score alpha production
mesh_tally = openmc.Tally(tally_id=1, name='alpha_production_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xa)']  # where X is a wild card


tallies = openmc.Tallies([mesh_tally])

Assigns a ring source of DT energy neutrons to the source.

This source has a 14MeV neutron energy, with a radius of 350cm and is half a ring (0 to 180 degrees)

If you are keen to make more realistic plasma sources take a looks at the openmc_plasma_source python package
https://github.com/fusion-energy/openmc-plasma-source

In [ ]:
my_source = openmc.IndependentSource()
my_source.angle = openmc.stats.Isotropic()
my_source.energy = openmc.stats.Discrete([14e6], [1])
my_source.space = openmc.stats.CylindricalIndependent(
    r=openmc.stats.Discrete([350], [1]),  # all source points at a radius of 350 cm
    phi=openmc.stats.Uniform(a=0, b=3.14), # angular distribution between 0 and pi
    z=openmc.stats.Discrete([0], [1]), # all source points at 0 on the z axis
    origin=(0.0, 0.0, 0.0), # centered around 0,0,0 x,y,z
)

Sets the simulation intensity then combines the materials, geometry, settines and tallies into a neutronics model and runs the simulation.

In [ ]:
settings = openmc.Settings()
settings.inactive = 0
settings.run_mode = 'fixed source'
settings.batches = 10
settings.particles = 2000
settings.source = my_source

my_model = openmc.Model(
    materials=materials,
    geometry=my_geometry,
    settings=settings,
    tallies=tallies
)

!rm *.h5  # just removes old summary.h5 or statepoint files
statepoint_file = my_model.run()

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint(statepoint_file)

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='alpha_production_on_mesh')

# converts the tally result into a VTK file
mesh.write_data_to_vtk(
    datasets={'mean': my_tally.mean},
    filename = "component_alpha_production_on_mesh.vtk",
)

The output file vtk file called "n_Xa_on_3D_mesh.vtk" should now appear in the left hand side sidebar. The file can be downloaded to your local computer by right mouse clicking on the file and selecting download.